<a href="https://colab.research.google.com/github/umersajid11/RAGAJKTourisam/blob/main/hackathonepecversion1_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio faiss-cpu sentence-transformers groq duckduckgo-search PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
!pip install --upgrade duckduckgo-search

In [7]:
# Install Required Libraries
#!pip install gradio faiss-cpu sentence-transformers groq duckduckgo-search PyPDF2

# Import Required Libraries
import os
import PyPDF2
import faiss
import numpy as np
from groq import Groq
from sentence_transformers import SentenceTransformer
#from duckduckgo_search import DuckDuckGoSearch
import requests
from bs4 import BeautifulSoup
import gradio as gr

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Load and Preprocess Dataset
def extract_text_from_pdf(pdf_path):
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Replace with the path to your dataset
pdf_text = extract_text_from_pdf("/content/Dataset.pdf")

# Chunk the dataset into manageable pieces
def chunk_text(text, chunk_size=512):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text)

# Generate embeddings for the chunks
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
chunk_embeddings = embedding_model.encode(chunks)

# Create a FAISS index for vector search
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

# Groq Llama-3-70B Query Function
def query_llama(prompt):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        stream=False,
    )
    return response.choices[0].message.content

# Search Engine Fallback with DuckDuckGo, Focused on AJK Tourism
def search_duckduckgo(query):
    ajk_query = f"{query} Azad Jammu and Kashmir tourism"
    url = f"https://html.duckduckgo.com/html/?q={ajk_query}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all("a", class_="result__snippet")
    snippets = [result.text for result in results if "Azad Jammu and Kashmir" in result.text]
    return " ".join(snippets[:3]) if snippets else "No relevant results found."

# Retrieve Context from Dataset
def retrieve_context(query, threshold=0.7):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), 1)
    if distances[0][0] < threshold:  # Check if similarity score meets threshold
        return chunks[indices[0][0]]
    return None

# Combined Retrieval-Augmented Generation Logic
def chatbot_response(query):
    # Check if the query matches any context in the dataset
    context = retrieve_context(query)
    if context:
        prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    else:
        # Fallback to DuckDuckGo Search
        web_context = search_duckduckgo(query)
        if web_context == "No relevant results found.":
            return "I'm sorry, I couldn't find relevant information about Azad Jammu and Kashmir tourism. Please try rephrasing your query."
        prompt = f"Using this information from the web: {web_context}\nQuestion: {query}\nAnswer:"
    return query_llama(prompt)

# Gradio Interface for the Chatbot
chat_history = []

def chat_interface(user_query):
    global chat_history
    bot_response = chatbot_response(user_query)
    chat_history.append((user_query, bot_response))
    return chat_history

with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown("### AJK Tourism Chatbot (Focused on Azad Jammu and Kashmir Tourism)")
    chatbot = gr.Chatbot(label="Chatbot Interaction")
    query = gr.Textbox(label="Ask Your Question about AJK Tourism")
    submit_button = gr.Button("Submit")
    submit_button.click(chat_interface, inputs=query, outputs=chatbot)

demo.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6dd8baabe7244419b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
